# 中文连续词袋模型 (CBOW) 实现

## 1. 导入必要的库

In [34]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
import jieba  # 用于中文分词

## 2. 准备中文数据

In [35]:
sentence = "自然语言处理是人工智能的一个重要分支。它涉及计算机对人类语言的理解和生成。"

# 1. 中文分词
seg_list = jieba.cut(sentence, cut_all=False)  # 使用精确模式分词
segmented_sentence = " ".join(seg_list)

print(f"分词结果: {segmented_sentence}")

# 2. 数据预处理
tokenizer = Tokenizer()
tokenizer.fit_on_texts([segmented_sentence])

word_index = tokenizer.word_index
vocabulary_size = len(word_index) + 1  # +1 for the padding token (if needed)

print(f"词汇表大小: {vocabulary_size}")
print(f"单词索引: {word_index}")

encoded_sentence = tokenizer.texts_to_sequences([segmented_sentence])[0]

分词结果: 自然语言 处理 是 人工智能 的 一个 重要 分支 。 它 涉及 计算机 对 人类 语言 的 理解 和 生成 。
词汇表大小: 19
单词索引: {'的': 1, '。': 2, '自然语言': 3, '处理': 4, '是': 5, '人工智能': 6, '一个': 7, '重要': 8, '分支': 9, '它': 10, '涉及': 11, '计算机': 12, '对': 13, '人类': 14, '语言': 15, '理解': 16, '和': 17, '生成': 18}


## 3. 创建训练数据 (CBOW 上下文-目标词对)

In [36]:
def generate_cbow_pairs(corpus, window_size):
    context_length = 2 * window_size
    cbow_pairs = []
    for i in range(window_size, len(corpus) - window_size):
        context_words = [corpus[i - j] for j in range(window_size, 0, -1)]  # 前面的上下文词
        context_words += [corpus[i + j] for j in range(1, window_size + 1)]  # 后面的上下文词
        target_word = corpus[i]
        cbow_pairs.append((context_words, target_word))
    return cbow_pairs

window_size = 2
cbow_pairs = generate_cbow_pairs(encoded_sentence, window_size)

# 分离上下文词和目标词
context_words, target_words = zip(*cbow_pairs)

# 转换为 NumPy 数组
context_words = np.array(context_words)
target_words = np.array(target_words)

print(f"生成的 CBOW 词对数量: {len(cbow_pairs)}")
print(f"第一个词对: context={context_words[0]}, target={target_words[0]}")

生成的 CBOW 词对数量: 16
第一个词对: context=[3 4 6 1], target=5


# 4. 构建 CBOW 模型

In [37]:
# 4. 构建 CBOW 模型
embedding_dim = 50  # 词嵌入维度
context_length = 2 * window_size # 上下文长度

cbow_input = keras.Input(shape=(context_length,), dtype='int32') # 输入是上下文词序列
embedding = layers.Embedding(vocabulary_size, embedding_dim)(cbow_input)
average = layers.GlobalAveragePooling1D()(embedding) # 对词嵌入求平均
output = layers.Dense(vocabulary_size, activation='softmax')(average) # 使用 softmax 进行多分类

cbow = keras.Model(inputs=cbow_input, outputs=output)

cbow.summary()

Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_10 (InputLayer)     │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_13 (Embedding)        │ (None, 4, 50)          │           950 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_10     │ (None, 50)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 19)             │           969 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,919 (7.50 KB)

 Trainable params: 1,919 (7.50 KB)

 Non-trainable params: 0 (0.00 B)

## 5. 编译和训练模型

In [38]:
# 5. 编译模型
cbow.compile(optimizer='adam',  # Adam 优化器
              loss='categorical_crossentropy',  # 适用于多分类问题
              metrics=['accuracy'])

# 将目标词转换为 one-hot 编码
target_words_one_hot = tf.keras.utils.to_categorical(target_words, num_classes=vocabulary_size)

print(f"context_words shape: {context_words.shape}")
print(f"target_words_one_hot shape: {target_words_one_hot.shape}")

context_words shape: (16, 4)
target_words_one_hot shape: (16, 19)


## 6. 训练模型

In [39]:
# 6. 训练模型
epochs = 100

cbow.fit(context_words, target_words_one_hot, epochs=epochs, batch_size=256, verbose=1)  # 开启 verbose，查看训练过程

print("训练完成!")

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.0000e+00 - loss: 2.9455
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.0000e+00 - loss: 2.9412
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - accuracy: 0.0625 - loss: 2.9369
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step - accuracy: 0.0625 - loss: 2.9326
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.1250 - loss: 2.9284
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.2500 - loss: 2.9241
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.3125 - loss: 2.9198
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.4375 - loss: 2.9155
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.5000 - loss: 2.9112
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.5625 - loss: 2.9069
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.5625 - loss: 2.9026
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.6

## 7. 获取词嵌入

In [40]:
# 7. 获取词嵌入
embedding_layer = cbow.layers[1]  # Embedding 层现在是第二层
embeddings = embedding_layer.get_weights()[0]

print(f"词嵌入矩阵的形状: {embeddings.shape}")  # (vocabulary_size, embedding_dim)

# 获取 "计算机" 的词嵌入
word = "计算机"
word_id = word_index[word]
embedding_vector = embeddings[word_id]

print(f"'{word}' 的词嵌入: {embedding_vector}")


词嵌入矩阵的形状: (19, 50)
'计算机' 的词嵌入: [-0.12553069  0.10213146  0.09594     0.15062957  0.14791958 -0.0935752
  0.21165147 -0.00703841  0.21184948 -0.16255817 -0.07577461  0.23779415
 -0.21095549  0.03916166 -0.06039023 -0.12436868 -0.17965183 -0.00406685
  0.00344226  0.12719455 -0.15322256  0.03567335  0.00178656 -0.1429299
  0.03928113  0.17985073  0.12053124 -0.20956117 -0.02914449  0.1490389
  0.04167986 -0.0372927  -0.12301474  0.11296449  0.14533854  0.13683203
  0.01195415 -0.14889722  0.0084296  -0.15990458  0.11018334  0.04844944
  0.03575176  0.00351978 -0.08600586 -0.12438011  0.1431647   0.20687006
 -0.04966841  0.01969392]
